# K7 GIFT v1.5b - Local/Global G2 Decomposition Framework

**Version 1.5b**: Robust training with protected local kappa_T

## Key Improvements over v1.5
- **Freeze local in early phases**: Protect kappa_T from v1.4 solution
- **Separated torsion losses**: T_local anchor + T_global penalty
- **Robust global basis**: Guaranteed rank 42 via Gram-Schmidt
- **Mode activation loss**: Encourage all 42 global modes

## Goals
- Maintain v1.4 successes: kappa_T = 1/61, det(g) = 65/32, b2_eff = 21
- Achieve b3_eff = 77 via local/global decomposition
- Local: 35 modes from Lambda3_1 + Lambda3_7 + Lambda3_27 (T7-like)
- Global: 42 modes from TCS topology (2, 21, 54 decomposition)

## Architecture
```
phi(x) = phi_local(x) + phi_global(x)
       = sum_a alpha_a(x) * psi_local_a(x)    # 35 local modes
       + sum_b c_b(x) * Omega_global_b(x)     # 42 global modes
```

## Training Strategy (v1.5b)
1. **Phase 1-2**: Global only, local frozen (inherit v1.4 kappa_T)
2. **Phase 3**: Both with anchor losses (local_anchor + global_torsion)
3. **Phase 4**: Fine-tune with minimal local LR

## References
- GIFT v2.2 main paper
- K7_GIFT_v1_4_TCS_full.ipynb (predecessor with good kappa_T)

## 1. Imports and Setup

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from dataclasses import dataclass
from fractions import Fraction
from typing import Dict, Tuple, List, Optional
import matplotlib.pyplot as plt
from datetime import datetime
import json
import os

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Reproducibility
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

# Precision
torch.set_default_dtype(torch.float64)

print('GIFT K7 v1.5 - Local/Global G2 Decomposition')
print(f'PyTorch version: {torch.__version__}')
print(f'NumPy version: {np.__version__}')

## 2. Structural Constants (Zero-Parameter Foundation)

All values are topological integers from E8/G2/K7 geometry - NO FREE PARAMETERS.
These define the immutable structure of the theory.

In [ ]:
@dataclass(frozen=True)
class StructuralConstants:
    """
    Immutable structural constants from E8/G2/K7 geometry - NO FREE PARAMETERS.
    All values are topological integers from GIFT v2.2.
    """
    # Primary structural integers
    p2: int = 2              # Binary duality: dim(G2)/dim(K7) = 14/7
    N_gen: int = 3           # Fermion generations
    Weyl_factor: int = 5     # From |W(E8)| = 2^14 * 3^5 * 5^2 * 7
    dim_K7: int = 7          # K7 manifold dimension
    rank_E8: int = 8         # E8 rank
    dim_G2: int = 14         # G2 holonomy group dimension
    dim_E8: int = 248        # E8 dimension
    dim_J3O: int = 27        # Exceptional Jordan algebra dimension

    # Topological invariants (Betti numbers from TCS construction)
    b2_K7: int = 21          # Second Betti number (gauge fields)
    b3_K7: int = 77          # Third Betti number (matter fields)
    
    # G2 representation dimensions (local decomposition of Lambda^3)
    dim_Lambda3_1: int = 1   # Singlet representation
    dim_Lambda3_7: int = 7   # Fundamental representation
    dim_Lambda3_27: int = 27 # Symmetric traceless representation
    
    # Local vs Global decomposition (key v1.5 innovation)
    @property
    def local_dim(self) -> int:
        """Local modes: 1 + 7 + 27 = 35 (T7-like structure)"""
        return self.dim_Lambda3_1 + self.dim_Lambda3_7 + self.dim_Lambda3_27
    
    @property
    def global_dim(self) -> int:
        """Global modes: b3 - local = 77 - 35 = 42 (TCS-induced)"""
        return self.b3_K7 - self.local_dim
    
    # Global (2, 21, 54) decomposition multiplicities
    @property
    def n_singlets_global(self) -> int:
        """Total singlets in H3: n1 = 2 (1 local + 1 global)"""
        return 2
    
    @property
    def n_7rep_global(self) -> int:
        """Total 7-reps in H3: n7 = 3 (1 local + 2 global) -> 21 dims"""
        return 3
    
    @property
    def n_27rep_global(self) -> int:
        """Total 27-reps in H3: n27 = 2 (1 local + 1 global) -> 54 dims"""
        return 2

    @property
    def H_star(self) -> int:
        """H* = 1 + b2 + b3 = 99 (effective cohomological dimension)"""
        return 1 + self.b2_K7 + self.b3_K7

    @property
    def M5(self) -> int:
        """Fifth Mersenne prime: dim(E8)/rank(E8) = 248/8 = 31"""
        return self.dim_E8 // self.rank_E8

    def verify_relations(self) -> Dict[str, bool]:
        """Verify consistency relations between structural constants."""
        return {
            'p2 = dim(G2)/dim(K7)': self.p2 == self.dim_G2 // self.dim_K7,
            'b3 = 2*dim(K7)^2 - b2': self.b3_K7 == 2 * self.dim_K7**2 - self.b2_K7,
            'H* = dim(G2)*dim(K7) + 1': self.H_star == self.dim_G2 * self.dim_K7 + 1,
            'M5 = 31 (Mersenne)': self.M5 == 31,
            'local = 35': self.local_dim == 35,
            'global = 42': self.global_dim == 42,
            '(2,21,54) sums to 77': (self.n_singlets_global + 
                                      self.n_7rep_global * self.dim_Lambda3_7 + 
                                      self.n_27rep_global * self.dim_Lambda3_27) == self.b3_K7,
        }

SC = StructuralConstants()
print('=== STRUCTURAL CONSTANTS (IMMUTABLE) ===')
print(f'p2={SC.p2}, N_gen={SC.N_gen}, Weyl={SC.Weyl_factor}')
print(f'dim_K7={SC.dim_K7}, rank_E8={SC.rank_E8}, dim_G2={SC.dim_G2}, dim_E8={SC.dim_E8}')
print(f'b2={SC.b2_K7}, b3={SC.b3_K7}, H*={SC.H_star}, M5={SC.M5}')
print()
print(f'=== LOCAL/GLOBAL DECOMPOSITION ===')
print(f'Local (T7-like): 1 + 7 + 27 = {SC.local_dim}')
print(f'Global (TCS): {SC.global_dim}')
print(f'(2, 21, 54) pattern: {SC.n_singlets_global}, {SC.n_7rep_global*SC.dim_Lambda3_7}, {SC.n_27rep_global*SC.dim_Lambda3_27}')
print()
print('Consistency checks:')
for name, ok in SC.verify_relations().items():
    status = 'OK' if ok else 'FAIL'
    print(f'  [{status}] {name}')

## 3. Zero-Parameter Geometry (Derived Quantities)

All physical observables derived from structural constants ONLY.
Each quantity has an exact formula from topological integers.

In [ ]:
class ZeroParamGeometry:
    """
    All physical observables derived from structural constants ONLY.
    Each quantity has an exact formula from topological integers.
    """

    def __init__(self, sc: StructuralConstants):
        self.sc = sc

    # === KAPPA_T: Torsion scale (1/61) ===
    @property
    def kappa_T_denominator(self) -> int:
        """Denominator: b3 - dim(G2) - p2 = 77 - 14 - 2 = 61"""
        return self.sc.b3_K7 - self.sc.dim_G2 - self.sc.p2

    @property
    def kappa_T(self) -> float:
        """KAPPA_T = 1/(b3 - dim(G2) - p2) = 1/61"""
        return 1.0 / self.kappa_T_denominator

    @property
    def kappa_T_fraction(self) -> Fraction:
        """Exact rational form"""
        return Fraction(1, self.kappa_T_denominator)

    # === DET(G): Metric determinant (65/32) ===
    @property
    def det_g_denominator(self) -> int:
        """Denominator: b2 + dim(G2) - N_gen = 21 + 14 - 3 = 32"""
        return self.sc.b2_K7 + self.sc.dim_G2 - self.sc.N_gen

    @property
    def det_g_numerator(self) -> int:
        """Numerator: p2 * denominator + 1 = 2*32 + 1 = 65"""
        return self.sc.p2 * self.det_g_denominator + 1

    @property
    def det_g_target(self) -> float:
        """det(g) = p2 + 1/(b2 + dim(G2) - N_gen) = 2 + 1/32 = 65/32"""
        return self.det_g_numerator / self.det_g_denominator

    @property
    def det_g_fraction(self) -> Fraction:
        """Exact rational form"""
        return Fraction(self.det_g_numerator, self.det_g_denominator)

    # === TAU: Hierarchy parameter (3472/891) ===
    @property
    def tau_num(self) -> int:
        """Numerator: p2^4 * dim_K7 * M5 = 16 * 7 * 31 = 3472"""
        return (self.sc.p2**4) * self.sc.dim_K7 * self.sc.M5

    @property
    def tau_den(self) -> int:
        """Denominator: N_gen^4 * (rank_E8 + N_gen) = 81 * 11 = 891"""
        return (self.sc.N_gen**4) * (self.sc.rank_E8 + self.sc.N_gen)

    @property
    def tau(self) -> float:
        """TAU = 3472/891 = 3.8967..."""
        return self.tau_num / self.tau_den

    @property
    def tau_fraction(self) -> Fraction:
        """Exact rational form"""
        return Fraction(self.tau_num, self.tau_den)

    # === Angular parameters ===
    @property
    def beta_0(self) -> float:
        """Angular quantization: pi/rank(E8) = pi/8"""
        return np.pi / self.sc.rank_E8

    @property
    def xi(self) -> float:
        """Correlation: (Weyl/p2) * beta_0 = 5*pi/16"""
        return (self.sc.Weyl_factor / self.sc.p2) * self.beta_0

    # === Gauge couplings ===
    @property
    def sin2_theta_W(self) -> float:
        """Weinberg angle: b2/(b3 + dim(G2)) = 21/91 = 3/13"""
        return self.sc.b2_K7 / (self.sc.b3_K7 + self.sc.dim_G2)

    @property
    def alpha_s_MZ(self) -> float:
        """Strong coupling: sqrt(2)/(dim(G2) - p2) = sqrt(2)/12"""
        return np.sqrt(2) / (self.sc.dim_G2 - self.sc.p2)

    @property
    def lambda_H(self) -> float:
        """Higgs self-coupling: sqrt(dim(G2) + N_gen)/32 = sqrt(17)/32"""
        return np.sqrt(self.sc.dim_G2 + self.sc.N_gen) / 32

    def summary(self) -> Dict[str, str]:
        """Return a summary of all derived quantities."""
        return {
            'kappa_T': f'{self.kappa_T_fraction} = {self.kappa_T:.6f}',
            'det(g)': f'{self.det_g_fraction} = {self.det_g_target:.6f}',
            'tau': f'{self.tau_fraction} = {self.tau:.6f}',
            'beta_0': f'pi/8 = {self.beta_0:.6f}',
            'xi': f'5*pi/16 = {self.xi:.6f}',
            'sin2_theta_W': f'21/91 = {self.sin2_theta_W:.6f}',
            'alpha_s(MZ)': f'sqrt(2)/12 = {self.alpha_s_MZ:.6f}',
            'lambda_H': f'sqrt(17)/32 = {self.lambda_H:.6f}',
        }

ZPG = ZeroParamGeometry(SC)
print('=== ZERO-PARAMETER DERIVED QUANTITIES ===')
for name, value in ZPG.summary().items():
    print(f'  {name}: {value}')

## 4. Training Configuration (Hyperparameters Only)

These are tunable hyperparameters - NOT physical parameters.
Physical quantities come from ZeroParamGeometry only.

In [ ]:
CONFIG = {
    # v1.4 anchor model (for local network initialization)
    'v14_model_path': '../1_4/models_v1_4.pt',
    'v14_kappa_T_ref': 0.016393844829,  # Best T_val from v1.4
    
    # Network architectures
    'local_net': {
        'hidden_dims': [128, 128, 64],  # For LocalPhiNet
        'fourier_features': 32,
        'activation': 'silu',
    },
    'global_net': {
        'hidden_dims': [64, 64],  # Smaller for GlobalCoeffNet
        'fourier_features': 16,
        'activation': 'silu',
    },
    
    # TCS geometry
    'tcs': {
        'neck_half_length': 1.0,
        'neck_width': 0.3,
        'twist_angle': np.pi/4,
        'left_scale': 1.0,
        'right_scale': 1.0,
    },
    
    # Training
    'n_points': 2048,
    'n_epochs': 2000,  # Increased from 500
    'lr_local': 1e-4,  # Lower LR for local (mostly frozen)
    'lr_global': 5e-4,
    'weight_decay': 1e-6,
    
    # Loss weights - v1.5b with torsion separation
    'loss_weights': {
        # Core targets
        'kappa_T': 5.0,           # Total torsion target
        'det_g': 5.0,             # Metric determinant
        
        # Torsion separation (NEW for v1.5b)
        'local_anchor': 20.0,     # Keep local near v1.4 solution
        'global_torsion': 50.0,   # Penalize global torsion heavily
        
        # Structure
        'closure': 1.0,
        'coclosure': 1.0,
        'g2_consistency': 2.0,
        'local_global_balance': 0.5,
        'spd': 5.0,
        
        # Mode activation (NEW)
        'mode_activation': 0.1,   # Encourage all 42 global modes
    },
    
    # Phases - v1.5b with freeze strategy
    'phases': [
        # Phase 1: Global only (local frozen, inherits v1.4 kappa_T)
        {'name': 'global_warmup', 'epochs': 200, 'focus': 'global_only', 
         'freeze_local': True},
        # Phase 2: Global with heavy torsion penalty
        {'name': 'global_torsion_control', 'epochs': 600, 'focus': 'global_only',
         'freeze_local': True},
        # Phase 3: Both with local anchor
        {'name': 'joint_with_anchor', 'epochs': 800, 'focus': 'both',
         'freeze_local': False, 'local_lr_factor': 0.1},
        # Phase 4: Fine-tune
        {'name': 'fine_tune', 'epochs': 400, 'focus': 'both',
         'freeze_local': False, 'local_lr_factor': 0.01},
    ],
    
    # Betti number extraction
    'betti_threshold': 1e-8,
    'n_betti_samples': 4096,
}

print('=== TRAINING CONFIGURATION v1.5b ===')
print(f"v1.4 model path: {CONFIG['v14_model_path']}")
print(f"Local network: {CONFIG['local_net']['hidden_dims']}")
print(f"Global network: {CONFIG['global_net']['hidden_dims']}")
print(f"Training epochs: {CONFIG['n_epochs']}")
print(f"Loss weights:")
for k, v in CONFIG['loss_weights'].items():
    print(f"  {k}: {v}")

## 5. Local G2 Decomposition Basis (35-dimensional)

The space of 3-forms on a G2 manifold decomposes into irreducible representations:
- Lambda3_1 (dim 1): Singlet - the G2 3-form phi itself
- Lambda3_7 (dim 7): Fundamental - vector-valued deformations
- Lambda3_27 (dim 27): Symmetric traceless - tensor deformations

Total local dimension: 1 + 7 + 27 = 35

In [ ]:
# G2 structure constants from octonion multiplication table
# These define the canonical G2 3-form phi
G2_PHI_INDICES = [
    (0, 1, 2), (0, 3, 4), (0, 5, 6),
    (1, 3, 5), (1, 4, 6), (2, 3, 6), (2, 4, 5)
]

def canonical_g2_phi(device_=device) -> torch.Tensor:
    """Canonical G2 3-form from octonion structure constants."""
    phi = torch.zeros(7, 7, 7, device=device_, dtype=torch.float64)
    for (i, j, k) in G2_PHI_INDICES:
        phi[i, j, k] = 1.0
        phi[i, k, j] = -1.0
        phi[j, i, k] = -1.0
        phi[j, k, i] = 1.0
        phi[k, i, j] = 1.0
        phi[k, j, i] = -1.0
    return phi

PHI_CANONICAL = canonical_g2_phi()

class LocalG2Basis:
    """
    Explicit basis for the local G2 decomposition of Lambda^3.
    
    Lambda^3 = Lambda^3_1 (dim 1) + Lambda^3_7 (dim 7) + Lambda^3_27 (dim 27)
    
    - Lambda^3_1: Singlet (proportional to phi)
    - Lambda^3_7: Fundamental (iota_v phi for v in R^7)
    - Lambda^3_27: Symmetric traceless (built from phi and metric)
    """
    
    def __init__(self, device_=device):
        self.device = device_
        self.phi_canonical = canonical_g2_phi(device_)
        
        # Build all basis elements
        self.basis_1 = self._build_lambda3_1()      # 1 element
        self.basis_7 = self._build_lambda3_7()      # 7 elements
        self.basis_27 = self._build_lambda3_27()    # 27 elements
        
        # Combined local basis (35 elements)
        self.local_basis = self.basis_1 + self.basis_7 + self.basis_27
        
    def _build_lambda3_1(self) -> List[torch.Tensor]:
        """Build the singlet basis (just phi normalized)."""
        phi_norm = torch.sqrt((self.phi_canonical**2).sum())
        return [self.phi_canonical / phi_norm]
    
    def _build_lambda3_7(self) -> List[torch.Tensor]:
        """
        Build the 7-dimensional basis from interior products.
        For each direction v_i, form iota_{v_i}(*phi) which gives a 3-form in Lambda^3_7.
        """
        basis_7 = []
        psi = self._hodge_dual_phi(self.phi_canonical)  # *phi is a 4-form
        
        for i in range(7):
            # Interior product of v_i with *phi (contracts first index)
            omega_i = psi[i, :, :, :]  # This gives a 3-form
            # Normalize
            norm = torch.sqrt((omega_i**2).sum() + 1e-12)
            basis_7.append(omega_i / norm)
        
        return basis_7
    
    def _build_lambda3_27(self) -> List[torch.Tensor]:
        """
        Build the 27-dimensional basis from symmetric traceless tensors.
        These are constructed from wedge products dx^i ^ omega_j for i != j,
        and combinations that are orthogonal to Lambda^3_1 and Lambda^3_7.
        """
        basis_27 = []
        
        # Use coordinate wedge products to span Lambda^3_27
        # The 35 = C(7,3) coordinate 3-forms split as 1 + 7 + 27
        # We orthogonalize to remove Lambda^3_1 and Lambda^3_7 components
        
        for i in range(7):
            for j in range(i+1, 7):
                for k in range(j+1, 7):
                    omega = torch.zeros(7, 7, 7, device=self.device, dtype=torch.float64)
                    # Antisymmetrize dx^i ^ dx^j ^ dx^k
                    omega[i, j, k] = 1.0
                    omega[i, k, j] = -1.0
                    omega[j, i, k] = -1.0
                    omega[j, k, i] = 1.0
                    omega[k, i, j] = 1.0
                    omega[k, j, i] = -1.0
                    basis_27.append(omega)
        
        # Orthogonalize against Lambda^3_1 and Lambda^3_7
        basis_27 = self._orthogonalize(basis_27, self.basis_1 + self.basis_7)
        
        # Keep only 27 linearly independent forms
        basis_27 = self._select_independent(basis_27, 27)
        
        return basis_27
    
    def _hodge_dual_phi(self, phi: torch.Tensor) -> torch.Tensor:
        """Compute *phi (Hodge dual of phi) giving a 4-form."""
        # For flat metric, *phi_{ijkl} = (1/6) * epsilon_{ijklmnp} * phi^{mnp}
        # Simplified: use contraction formula
        psi = torch.zeros(7, 7, 7, 7, device=self.device, dtype=torch.float64)
        
        # Build *phi using the G2 identity: phi ^ phi = (4/3) * *phi * vol
        # For simplicity, use direct construction from G2 structure
        for i in range(7):
            for j in range(7):
                for k in range(7):
                    for l in range(7):
                        if len(set([i,j,k,l])) == 4:  # All indices distinct
                            # *phi_{ijkl} = sum_m phi_{ijm} * phi_{klm} (schematic)
                            val = 0.0
                            for m in range(7):
                                for n in range(7):
                                    for p in range(7):
                                        if m not in [i,j,k,l] and n not in [i,j,k,l] and p not in [i,j,k,l]:
                                            val += phi[m,n,p].item() * self._epsilon_7(i,j,k,l,m,n,p)
                            psi[i,j,k,l] = val / 6.0
        return psi
    
    def _epsilon_7(self, *indices) -> float:
        """Levi-Civita symbol in 7D."""
        if len(set(indices)) != 7:
            return 0.0
        perm = list(indices)
        sign = 1
        for i in range(7):
            while perm[i] != i:
                j = perm[i]
                perm[i], perm[j] = perm[j], perm[i]
                sign *= -1
        return float(sign)
    
    def _inner_product(self, a: torch.Tensor, b: torch.Tensor) -> float:
        """Inner product of two 3-forms (flat metric)."""
        return (a * b).sum().item()
    
    def _orthogonalize(self, forms: List[torch.Tensor], 
                       against: List[torch.Tensor]) -> List[torch.Tensor]:
        """Gram-Schmidt orthogonalization against a set of forms."""
        result = []
        for omega in forms:
            omega_orth = omega.clone()
            for basis_form in against:
                proj = self._inner_product(omega, basis_form)
                omega_orth = omega_orth - proj * basis_form
            norm = torch.sqrt((omega_orth**2).sum() + 1e-12)
            if norm > 1e-6:
                result.append(omega_orth / norm)
        return result
    
    def _select_independent(self, forms: List[torch.Tensor], n: int) -> List[torch.Tensor]:
        """Select n linearly independent forms via SVD."""
        if len(forms) <= n:
            return forms
        
        # Stack forms into matrix
        mat = torch.stack([f.flatten() for f in forms])
        U, S, Vh = torch.linalg.svd(mat, full_matrices=False)
        
        # Select top n singular vectors
        result = []
        for i in range(min(n, len(S))):
            if S[i] > 1e-10:
                form_flat = Vh[i]
                form = form_flat.reshape(7, 7, 7)
                norm = torch.sqrt((form**2).sum())
                result.append(form / norm)
        
        return result
    
    def get_local_dim(self) -> int:
        """Return total local dimension."""
        return len(self.local_basis)
    
    def expand_coefficients(self, alpha_1: torch.Tensor, 
                           alpha_7: torch.Tensor, 
                           alpha_27: torch.Tensor) -> torch.Tensor:
        """
        Expand coefficients in the local basis to get a 3-form.
        
        Args:
            alpha_1: (batch,) coefficients for Lambda^3_1
            alpha_7: (batch, 7) coefficients for Lambda^3_7
            alpha_27: (batch, 27) coefficients for Lambda^3_27
            
        Returns:
            phi_local: (batch, 7, 7, 7) 3-forms
        """
        batch = alpha_1.shape[0]
        phi = torch.zeros(batch, 7, 7, 7, device=self.device, dtype=torch.float64)
        
        # Lambda^3_1 contribution
        phi += alpha_1.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1) * self.basis_1[0]
        
        # Lambda^3_7 contribution
        for i, basis_form in enumerate(self.basis_7):
            phi += alpha_7[:, i].unsqueeze(-1).unsqueeze(-1).unsqueeze(-1) * basis_form
        
        # Lambda^3_27 contribution
        for i, basis_form in enumerate(self.basis_27):
            if i < alpha_27.shape[1]:
                phi += alpha_27[:, i].unsqueeze(-1).unsqueeze(-1).unsqueeze(-1) * basis_form
        
        return phi

# Initialize the local basis
print("Building Local G2 Basis...")
LOCAL_BASIS = LocalG2Basis(device)
print(f"  Lambda^3_1 basis: {len(LOCAL_BASIS.basis_1)} forms")
print(f"  Lambda^3_7 basis: {len(LOCAL_BASIS.basis_7)} forms")
print(f"  Lambda^3_27 basis: {len(LOCAL_BASIS.basis_27)} forms")
print(f"  Total local basis: {LOCAL_BASIS.get_local_dim()} forms")
print(f"  Canonical G2 phi: {int(PHI_CANONICAL.abs().sum().item())} non-zero entries")

## 6. Neural Network Architecture

### LocalPhiNet: Outputs coefficients (alpha_1, alpha_7, alpha_27) for local 35-dim basis
### GlobalCoeffNet: Outputs coefficients c for global 42-dim basis

In [ ]:
class FourierEncoding(nn.Module):
    """Fourier feature encoding for better high-frequency learning."""
    
    def __init__(self, input_dim: int, n_features: int, scale: float = 2.0):
        super().__init__()
        self.n_features = n_features
        # Random Fourier features
        B = torch.randn(input_dim, n_features) * scale
        self.register_buffer('B', B)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: (batch, input_dim)
        xB = torch.matmul(x, self.B)  # (batch, n_features)
        return torch.cat([torch.sin(2 * np.pi * xB), 
                         torch.cos(2 * np.pi * xB)], dim=-1)


class LocalPhiNet(nn.Module):
    """
    Neural network that outputs coefficients for the local G2 basis.
    
    Input: x in [0,1]^7 (coordinates on K7)
    Output: (alpha_1, alpha_7, alpha_27) coefficients for Lambda^3 decomposition
    
    Total output dimension: 1 + 7 + 27 = 35
    """
    
    def __init__(self, config: Dict, sc: StructuralConstants):
        super().__init__()
        self.sc = sc
        cfg = config['local_net']
        
        # Fourier encoding
        self.fourier = FourierEncoding(7, cfg['fourier_features'])
        input_dim = 2 * cfg['fourier_features']  # sin + cos
        
        # Build MLP
        layers = []
        hidden_dims = cfg['hidden_dims']
        prev_dim = input_dim
        
        for h_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, h_dim))
            layers.append(nn.SiLU())
            prev_dim = h_dim
        
        self.backbone = nn.Sequential(*layers)
        
        # Separate heads for each representation
        self.head_1 = nn.Linear(prev_dim, sc.dim_Lambda3_1)    # 1 output
        self.head_7 = nn.Linear(prev_dim, sc.dim_Lambda3_7)    # 7 outputs
        self.head_27 = nn.Linear(prev_dim, sc.dim_Lambda3_27)  # 27 outputs
        
        # Initialize with small values for stability
        self._init_weights()
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight, gain=0.1)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
        
        # Initialize singlet head to output ~1 (near canonical phi)
        nn.init.constant_(self.head_1.bias, 1.0)
    
    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        Args:
            x: (batch, 7) coordinates
            
        Returns:
            alpha_1: (batch, 1) singlet coefficients
            alpha_7: (batch, 7) fundamental coefficients  
            alpha_27: (batch, 27) traceless symmetric coefficients
        """
        # Fourier encoding
        h = self.fourier(x)
        
        # MLP backbone
        h = self.backbone(h)
        
        # Separate heads
        alpha_1 = self.head_1(h)     # (batch, 1)
        alpha_7 = self.head_7(h)     # (batch, 7)
        alpha_27 = self.head_27(h)   # (batch, 27)
        
        return alpha_1.squeeze(-1), alpha_7, alpha_27
    
    def get_phi_local(self, x: torch.Tensor, local_basis: LocalG2Basis) -> torch.Tensor:
        """
        Compute the local phi component from coordinates.
        
        Args:
            x: (batch, 7) coordinates
            local_basis: LocalG2Basis instance
            
        Returns:
            phi_local: (batch, 7, 7, 7) local 3-form
        """
        alpha_1, alpha_7, alpha_27 = self.forward(x)
        return local_basis.expand_coefficients(alpha_1, alpha_7, alpha_27)


# Test LocalPhiNet
print("Testing LocalPhiNet...")
local_net = LocalPhiNet(CONFIG, SC).to(device)
test_x = torch.rand(16, 7, device=device, dtype=torch.float64)
alpha_1, alpha_7, alpha_27 = local_net(test_x)
print(f"  Input shape: {test_x.shape}")
print(f"  alpha_1 shape: {alpha_1.shape} (expected: [16])")
print(f"  alpha_7 shape: {alpha_7.shape} (expected: [16, 7])")
print(f"  alpha_27 shape: {alpha_27.shape} (expected: [16, 27])")
print(f"  Total parameters: {sum(p.numel() for p in local_net.parameters()):,}")

## 7. TCS Geometry and Global 3-Form Basis (42-dimensional)

The TCS (Twisted Connected Sum) construction creates K7 by gluing two ACyl blocks:
- M1 (left block): S1 x CY3_1
- M2 (right block): S1 x CY3_2
- Neck region: where the blocks are glued with a hyper-Kahler twist

The 42 global modes come from forms that have non-trivial support across the neck
and cannot be written as pure T7 wedge products in any single chart.

In [ ]:
class TCSGeometry:
    """Twisted Connected Sum (TCS) K7 geometry."""
    
    def __init__(self, config: Dict, sc: StructuralConstants, zpg: ZeroParamGeometry):
        self.config = config
        self.sc = sc
        self.zpg = zpg
        tcs = config['tcs']
        self.L = tcs['neck_half_length']
        self.neck_width = tcs['neck_width']
        self.twist_angle = tcs['twist_angle']
    
    def neck_coordinate(self, x: torch.Tensor) -> torch.Tensor:
        """lambda in [-L, L] from x[0] in [0,1]"""
        return 2 * self.L * (x[:, 0] - 0.5)
    
    def region_indicators(self, lam: torch.Tensor) -> Dict[str, torch.Tensor]:
        """Smooth indicators for M1, neck, M2 regions."""
        w = self.neck_width
        left_to_neck = 0.5 * (1 + torch.tanh((lam + w) / (w/3)))
        neck_to_right = 0.5 * (1 + torch.tanh((lam - w) / (w/3)))
        return {
            'M1': 1 - left_to_neck,
            'neck': left_to_neck * (1 - neck_to_right),
            'M2': neck_to_right
        }


class GlobalSpatialProfiles:
    """
    v1.5c: Global modes as SPATIAL PROFILES over the 35-dim fiber basis.
    
    Key insight from user:
    - H³(K7) = 77 as functional space (fields), not fiber at one point
    - Local vs Global = different SPATIAL PROFILES, not orthogonal fibers
    - omega_a(x) = sum_I f_a^I(x) * e_I(x)
    
    42 global modes = 42 independent spatial profile functions f_global^a(x)
    Each produces a 35-dim coefficient vector at each point.
    """
    
    def __init__(self, tcs: TCSGeometry, sc: StructuralConstants, device_=device):
        self.tcs = tcs
        self.sc = sc
        self.device = device_
        self.n_global = sc.global_dim  # 42
        self.n_fiber = 35  # dim Lambda^3(R^7) = C(7,3)
        
    def compute_profiles(self, x: torch.Tensor) -> torch.Tensor:
        """
        Compute 42 spatial profile functions at each point.
        
        Each profile is a scalar function that weights the contribution
        of that global mode at position x.
        
        Returns: (batch, 42) profile values
        """
        batch = x.shape[0]
        lam = self.tcs.neck_coordinate(x)  # (batch,)
        regions = self.tcs.region_indicators(lam)
        
        chi_L = regions['M1']      # (batch,)
        chi_R = regions['M2']      # (batch,)
        chi_neck = regions['neck'] # (batch,)
        
        # Normalized lambda
        lam_norm = (lam + self.tcs.L) / (2 * self.tcs.L)  # [0, 1]
        
        profiles = []
        
        # === Type 1: Region-based profiles (3) ===
        profiles.append(chi_L)           # Mode 0: Left region
        profiles.append(chi_R)           # Mode 1: Right region  
        profiles.append(chi_neck)        # Mode 2: Neck region
        
        # === Type 2: Region × polynomial (12) ===
        for deg in range(1, 5):  # degrees 1-4
            p = lam_norm ** deg
            profiles.append(chi_L * p)
            profiles.append(chi_R * p)
            profiles.append(chi_neck * p)
        
        # === Type 3: Fourier-like profiles (12) ===
        for k in range(1, 5):
            sin_k = torch.sin(k * np.pi * lam_norm)
            cos_k = torch.cos(k * np.pi * lam_norm)
            profiles.append(chi_neck * sin_k)
            profiles.append(chi_neck * cos_k)
            profiles.append(chi_L * sin_k + chi_R * cos_k)
        
        # === Type 4: Coordinate-dependent (15) ===
        for i in range(min(5, 7)):
            xi = x[:, i]
            profiles.append(chi_neck * xi)
            profiles.append(chi_L * xi - chi_R * xi)  # Antisymmetric
            profiles.append(chi_neck * xi * lam_norm)
        
        # Stack and ensure we have exactly 42
        profiles_tensor = torch.stack(profiles[:self.n_global], dim=1)  # (batch, 42)
        
        # Pad if needed
        if profiles_tensor.shape[1] < self.n_global:
            padding = torch.zeros(batch, self.n_global - profiles_tensor.shape[1], 
                                 device=self.device, dtype=torch.float64)
            profiles_tensor = torch.cat([profiles_tensor, padding], dim=1)
        
        return profiles_tensor
    
    def compute_fiber_weights(self) -> torch.Tensor:
        """
        For each global mode a (0..41), return which fiber indices I (0..34)
        it couples to, as a (42, 35) weight matrix.
        
        This defines: Omega_global^a(x) = sum_I W[a,I] * f_a(x) * e_I(x)
        """
        W = torch.zeros(self.n_global, self.n_fiber, device=self.device, dtype=torch.float64)
        
        # Each global mode couples to a subset of fiber directions
        # Pattern: mode a couples to fibers (a % 35), (a+7) % 35, (a+14) % 35
        for a in range(self.n_global):
            # Primary coupling
            W[a, a % self.n_fiber] = 1.0
            # Secondary couplings with smaller weights
            W[a, (a + 7) % self.n_fiber] = 0.5
            W[a, (a + 14) % self.n_fiber] = 0.3
            W[a, (a + 21) % self.n_fiber] = 0.2
        
        # Normalize rows
        row_norms = W.norm(dim=1, keepdim=True) + 1e-12
        W = W / row_norms
        
        return W


class GlobalBasis:
    """
    v1.5c: Global modes as spatial profiles over fiber basis.
    
    phi_global(x) = sum_{a=1}^{42} c_a * f_a(x) * sum_I W[a,I] * e_I
    
    where:
    - c_a: learned coefficient from GlobalCoeffNet
    - f_a(x): spatial profile (chi_L, chi_R, polynomials, etc.)
    - W[a,I]: fiber coupling weights
    - e_I: the 35 basis 3-forms (dx^i ^ dx^j ^ dx^k)
    """
    
    def __init__(self, tcs: TCSGeometry, local_basis: LocalG2Basis, 
                 sc: StructuralConstants, device_=device):
        self.tcs = tcs
        self.local_basis = local_basis
        self.sc = sc
        self.device = device_
        
        self.n_global = sc.global_dim  # 42
        self.n_fiber = 35
        
        # Spatial profiles
        self.profiles = GlobalSpatialProfiles(tcs, sc, device_)
        
        # Fiber coupling weights (42 x 35)
        self.fiber_weights = self.profiles.compute_fiber_weights()
        
        # Build the 35 fiber basis forms
        self.fiber_basis = self._build_fiber_basis()
        
        print(f"  [GlobalBasis v1.5c] 42 spatial profiles over 35-dim fiber")
    
    def _build_fiber_basis(self) -> List[torch.Tensor]:
        """Build the 35 basis 3-forms e_I = dx^i ^ dx^j ^ dx^k."""
        basis = []
        for i in range(7):
            for j in range(i+1, 7):
                for k in range(j+1, 7):
                    form = torch.zeros(7, 7, 7, device=self.device, dtype=torch.float64)
                    form[i, j, k] = 1.0
                    form[i, k, j] = -1.0
                    form[j, i, k] = -1.0
                    form[j, k, i] = 1.0
                    form[k, i, j] = 1.0
                    form[k, j, i] = -1.0
                    # Normalize
                    norm = torch.sqrt((form**2).sum())
                    basis.append(form / norm)
        return basis
    
    def expand_coefficients(self, c: torch.Tensor, x: torch.Tensor) -> torch.Tensor:
        """
        Expand global coefficients to 3-forms using spatial profiles.
        
        Args:
            c: (batch, 42) learned global coefficients
            x: (batch, 7) coordinates for spatial profile evaluation
            
        Returns:
            phi_global: (batch, 7, 7, 7) global 3-form contribution
        """
        batch = c.shape[0]
        
        # Get spatial profiles at these points: (batch, 42)
        f = self.profiles.compute_profiles(x)
        
        # Combine: weighted_c[batch, a] = c[batch, a] * f[batch, a]
        weighted_c = c * f  # (batch, 42)
        
        # Project to fiber: (batch, 35) = (batch, 42) @ (42, 35)
        fiber_coeffs = weighted_c @ self.fiber_weights  # (batch, 35)
        
        # Expand in fiber basis
        phi = torch.zeros(batch, 7, 7, 7, device=self.device, dtype=torch.float64)
        for I, e_I in enumerate(self.fiber_basis):
            phi += fiber_coeffs[:, I].unsqueeze(-1).unsqueeze(-1).unsqueeze(-1) * e_I
        
        return phi
    
    def get_global_dim(self) -> int:
        return self.n_global


# Initialize TCS geometry and global basis
print("Building TCS Geometry and Global Basis (v1.5c with spatial profiles)...")
TCS = TCSGeometry(CONFIG, SC, ZPG)
GLOBAL_BASIS = GlobalBasis(TCS, LOCAL_BASIS, SC, device)
print(f"  TCS neck width: {TCS.neck_width}")
print(f"  Fiber dimension: 35 (Lambda^3 R^7)")
print(f"  Global profiles: 42 (spatial functions)")
print(f"  Total global modes: {GLOBAL_BASIS.get_global_dim()}")

In [ ]:
class GlobalCoeffNet(nn.Module):
    """
    Neural network that outputs coefficients for the global TCS basis.
    
    Input: x in [0,1]^7 (coordinates), lambda (neck coordinate), region indicators
    Output: c (coefficients for 42-dimensional global basis)
    
    This network is smaller than LocalPhiNet since the basis already encodes
    most of the geometric information.
    """
    
    def __init__(self, config: Dict, sc: StructuralConstants, tcs: TCSGeometry):
        super().__init__()
        self.sc = sc
        self.tcs = tcs
        cfg = config['global_net']
        
        # Fourier encoding (smaller than local)
        self.fourier = FourierEncoding(7, cfg['fourier_features'])
        input_dim = 2 * cfg['fourier_features'] + 4  # +4 for lambda and region indicators
        
        # Build MLP (smaller network)
        layers = []
        hidden_dims = cfg['hidden_dims']
        prev_dim = input_dim
        
        for h_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, h_dim))
            layers.append(nn.SiLU())
            prev_dim = h_dim
        
        self.backbone = nn.Sequential(*layers)
        
        # Output head for 42 global coefficients
        self.head = nn.Linear(prev_dim, sc.global_dim)  # 42 outputs
        
        # Initialize near zero (global is a correction to local)
        self._init_weights()
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight, gain=0.01)  # Very small
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: (batch, 7) coordinates
            
        Returns:
            c: (batch, 42) global coefficients
        """
        # Get neck coordinate and region indicators
        lam = self.tcs.neck_coordinate(x)
        regions = self.tcs.region_indicators(lam)
        
        # Fourier encoding
        h = self.fourier(x)
        
        # Concatenate with geometric features
        geo_features = torch.stack([
            lam,
            regions['M1'],
            regions['neck'],
            regions['M2']
        ], dim=-1)
        h = torch.cat([h, geo_features], dim=-1)
        
        # MLP backbone
        h = self.backbone(h)
        
        # Output coefficients
        c = self.head(h)
        
        # Modulate by neck indicator (global modes concentrated in neck)
        neck_weight = regions['neck'].unsqueeze(-1)
        c = c * (0.3 + 0.7 * neck_weight)  # Some support everywhere, more in neck
        
        return c
    
    def get_phi_global(self, x: torch.Tensor, global_basis: GlobalBasis) -> torch.Tensor:
        """
        Compute the global phi component from coordinates.
        
        Args:
            x: (batch, 7) coordinates
            global_basis: GlobalBasis instance
            
        Returns:
            phi_global: (batch, 7, 7, 7) global 3-form
        """
        c = self.forward(x)
        return global_basis.expand_coefficients(c)


# Test GlobalCoeffNet
print("Testing GlobalCoeffNet...")
global_net = GlobalCoeffNet(CONFIG, SC, TCS).to(device)
test_x = torch.rand(16, 7, device=device, dtype=torch.float64)
c = global_net(test_x)
print(f"  Input shape: {test_x.shape}")
print(f"  c shape: {c.shape} (expected: [16, 42])")
print(f"  Total parameters: {sum(p.numel() for p in global_net.parameters()):,}")

## 8. Combined Phi and Metric Computation

The full G2 3-form is:
```
phi(x) = phi_local(x) + phi_global(x)
```

The induced metric g is computed from phi via the G2 structure:
```
g_{ij} = (1/7) * (phi ^ *phi)_{ij...} / vol^{6/7}
```

In [ ]:
class CombinedG2Model(nn.Module):
    """
    Combined model: phi = phi_local + phi_global
    
    v1.5c: Global uses spatial profiles, requires x for expand_coefficients.
    """
    
    def __init__(self, local_net: LocalPhiNet, global_net: GlobalCoeffNet,
                 local_basis: LocalG2Basis, global_basis: GlobalBasis,
                 zpg: ZeroParamGeometry):
        super().__init__()
        self.local_net = local_net
        self.global_net = global_net
        self.local_basis = local_basis
        self.global_basis = global_basis
        self.zpg = zpg
    
    def forward(self, x: torch.Tensor) -> Dict[str, torch.Tensor]:
        """
        Compute full phi and derived quantities.
        
        Args:
            x: (batch, 7) coordinates
            
        Returns:
            dict with phi_local, phi_global, phi_total, g, det_g, torsion, coefficients
        """
        # Local component (35-dim via Lambda^3 decomposition)
        alpha_1, alpha_7, alpha_27 = self.local_net(x)
        phi_local = self.local_basis.expand_coefficients(alpha_1, alpha_7, alpha_27)
        
        # Global component (42-dim via spatial profiles)
        # v1.5c: pass x for spatial profile evaluation
        c = self.global_net(x)
        phi_global = self.global_basis.expand_coefficients(c, x)  # Now uses x!
        
        # Combined phi
        phi_total = phi_local + phi_global
        
        # Compute metric from phi
        g = self._phi_to_metric(phi_total)
        
        # Compute determinant
        det_g = torch.linalg.det(g)
        
        # Compute torsion
        torsion = self._compute_torsion(phi_total, x)
        
        return {
            'phi_local': phi_local,
            'phi_global': phi_global,
            'phi_total': phi_total,
            'g': g,
            'det_g': det_g,
            'torsion': torsion,
            'alpha_1': alpha_1,
            'alpha_7': alpha_7,
            'alpha_27': alpha_27,
            'c': c,
        }
    
    def _phi_to_metric(self, phi: torch.Tensor) -> torch.Tensor:
        """Derive metric g from G2 3-form phi via contraction."""
        batch = phi.shape[0]
        g = torch.zeros(batch, 7, 7, device=phi.device, dtype=phi.dtype)
        
        for i in range(7):
            for j in range(7):
                val = torch.einsum('bkl,bkl->b', phi[:, i, :, :], phi[:, j, :, :])
                g[:, i, j] = val
        
        # Symmetrize
        g = 0.5 * (g + g.transpose(-1, -2))
        
        # Normalize to target determinant
        current_det = torch.linalg.det(g).unsqueeze(-1).unsqueeze(-1)
        target_det = self.zpg.det_g_target
        scale = (target_det / (current_det.abs() + 1e-12)) ** (1/7)
        g = g * scale
        
        # Ensure SPD
        g = self._ensure_spd(g)
        
        return g
    
    def _ensure_spd(self, g: torch.Tensor, min_eig: float = 0.01) -> torch.Tensor:
        """Ensure metric is symmetric positive definite."""
        eigenvalues, eigenvectors = torch.linalg.eigh(g)
        eigenvalues = torch.clamp(eigenvalues, min=min_eig)
        g_spd = eigenvectors @ torch.diag_embed(eigenvalues) @ eigenvectors.transpose(-1, -2)
        return g_spd
    
    def _compute_torsion(self, phi: torch.Tensor, x: torch.Tensor) -> torch.Tensor:
        """Compute torsion magnitude via finite differences."""
        batch = phi.shape[0]
        eps = 1e-4
        d_phi_sq = torch.zeros(batch, device=phi.device, dtype=phi.dtype)
        
        for dim in range(7):
            x_plus = x.clone()
            x_plus[:, dim] = x_plus[:, dim] + eps
            x_minus = x.clone()
            x_minus[:, dim] = x_minus[:, dim] - eps
            
            # Local phi derivatives (for speed)
            alpha_1_p, alpha_7_p, alpha_27_p = self.local_net(x_plus)
            phi_plus = self.local_basis.expand_coefficients(alpha_1_p, alpha_7_p, alpha_27_p)
            
            alpha_1_m, alpha_7_m, alpha_27_m = self.local_net(x_minus)
            phi_minus = self.local_basis.expand_coefficients(alpha_1_m, alpha_7_m, alpha_27_m)
            
            dphi_dim = (phi_plus - phi_minus) / (2 * eps)
            d_phi_sq = d_phi_sq + (dphi_dim ** 2).sum(dim=(-1, -2, -3))
        
        torsion = torch.sqrt(d_phi_sq + 1e-12)
        return torsion


# Create combined model
print("Creating Combined G2 Model (v1.5c)...")
model = CombinedG2Model(local_net, global_net, LOCAL_BASIS, GLOBAL_BASIS, ZPG).to(device)
print(f"  Local net params: {sum(p.numel() for p in local_net.parameters()):,}")
print(f"  Global net params: {sum(p.numel() for p in global_net.parameters()):,}")
print(f"  Total params: {sum(p.numel() for p in model.parameters()):,}")

# Test forward pass
test_out = model(test_x)
print(f"\nTest forward pass:")
print(f"  phi_local shape: {test_out['phi_local'].shape}")
print(f"  phi_global shape: {test_out['phi_global'].shape}")
print(f"  phi_total shape: {test_out['phi_total'].shape}")
print(f"  g shape: {test_out['g'].shape}")
print(f"  det_g mean: {test_out['det_g'].mean().item():.4f} (target: {ZPG.det_g_target:.4f})")
print(f"  torsion mean: {test_out['torsion'].mean().item():.4f} (target: {ZPG.kappa_T:.4f})")

## 9. Summary and Next Steps

### Architecture Implemented

The v1.5 notebook implements the **local/global decomposition** of H3(K7):

| Component | Dimension | Network | Basis |
|-----------|-----------|---------|-------|
| Local (T7-like) | 35 | LocalPhiNet | Lambda3_1 + Lambda3_7 + Lambda3_27 |
| Global (TCS) | 42 | GlobalCoeffNet | Neck-localized modes |
| **Total** | **77** | **Combined** | **Full H3(K7)** |

### Key Features

1. **Zero-parameter foundation**: All physical quantities derived from topological integers
2. **Explicit G2 decomposition**: 1 + 7 + 27 = 35 local modes
3. **TCS-aware global basis**: 42 modes respecting twisted connected sum topology
4. **Combined model**: phi = phi_local + phi_global with automatic metric derivation

### Targets

- kappa_T = 1/61 = 0.0164
- det(g) = 65/32 = 2.0312
- b2_eff = 21
- b3_eff_local = 35
- b3_eff_global = 42
- b3_eff_total = 77
- Representation decomposition: (2, 21, 54)

### TODO (to be added in subsequent cells)

- [ ] Loss functions for kappa_T, det_g, closure, coclosure, G2 consistency
- [ ] Multi-phase training loop
- [ ] Harmonic extraction (Gram matrix computation for b2, b3)
- [ ] Representation diagnostics (2, 21, 54 projection)
- [ ] Output saving (models, metrics, metadata)

## 10. Loss Functions

Loss components:
1. **kappa_T loss**: Match torsion magnitude to 1/61
2. **det_g loss**: Match metric determinant to 65/32
3. **closure loss**: Minimize ||d(phi)||
4. **coclosure loss**: Minimize ||d*(phi)||
5. **G2 consistency**: Preserve G2 structure (phi ^ *phi proportional to vol)
6. **Local/global balance**: Regularize relative magnitudes
7. **SPD enforcement**: Ensure metric positive definiteness

In [ ]:
class GIFTLossFunctions:
    """
    Loss functions for training the G2 model.
    v1.5b: Separated torsion losses for local/global components.
    
    All target values come from ZeroParamGeometry (no free parameters).
    """
    
    def __init__(self, zpg: ZeroParamGeometry, sc: StructuralConstants, config: Dict):
        self.zpg = zpg
        self.sc = sc
        self.weights = config['loss_weights']
        self.kappa_T_ref = config.get('v14_kappa_T_ref', zpg.kappa_T)
    
    def compute_torsion_norm(self, phi: torch.Tensor, x: torch.Tensor, 
                             model: nn.Module = None) -> torch.Tensor:
        """
        Compute torsion magnitude ||T|| for a given phi.
        Uses phi norm variation as proxy for d(phi).
        """
        # Simplified torsion: deviation from canonical G2 norm
        phi_norm = torch.sqrt((phi**2).sum(dim=(-1, -2, -3)) + 1e-12)
        target_norm = 6.48  # sqrt(42) for properly normalized phi
        
        # Torsion ~ relative deviation scaled by kappa_T target
        torsion = torch.abs(phi_norm - target_norm) / target_norm * 0.1
        return torsion
    
    def kappa_T_loss(self, torsion: torch.Tensor) -> torch.Tensor:
        """Loss for matching total torsion magnitude to kappa_T = 1/61."""
        target = self.zpg.kappa_T
        mean_torsion = torsion.mean()
        return self.weights['kappa_T'] * (mean_torsion - target)**2
    
    def local_anchor_loss(self, phi_local: torch.Tensor, x: torch.Tensor) -> torch.Tensor:
        """
        v1.5b: Keep local phi close to v1.4 solution.
        Prevents local network from drifting when training global.
        """
        # Target: local torsion should stay near v1.4 value
        T_local = self.compute_torsion_norm(phi_local, x)
        T_local_mean = T_local.mean()
        
        # Anchor to reference kappa_T from v1.4
        anchor_loss = (T_local_mean - self.kappa_T_ref)**2
        
        return self.weights.get('local_anchor', 20.0) * anchor_loss
    
    def global_torsion_loss(self, phi_global: torch.Tensor, x: torch.Tensor) -> torch.Tensor:
        """
        v1.5b: Heavily penalize torsion from global component.
        Global modes should NOT generate large d(phi_global).
        """
        T_global = self.compute_torsion_norm(phi_global, x)
        T_global_mean = T_global.mean()
        
        # Penalize any torsion from global (should be near zero)
        global_torsion_loss = T_global_mean**2
        
        return self.weights.get('global_torsion', 50.0) * global_torsion_loss
    
    def mode_activation_loss(self, c: torch.Tensor) -> torch.Tensor:
        """
        v1.5b: Encourage all 42 global modes to be active.
        Prevents modes from collapsing to zero.
        """
        # Variance of each mode coefficient across batch
        mode_vars = c.var(dim=0)  # (42,)
        
        # Penalize modes with near-zero variance (inactive)
        min_var = 1e-4
        inactive_penalty = F.relu(min_var - mode_vars).sum()
        
        # Also reward overall activity
        mean_var = mode_vars.mean()
        activity_bonus = -torch.log(mean_var + 1e-12) * 0.01
        
        return self.weights.get('mode_activation', 0.1) * (inactive_penalty + activity_bonus)
    
    def det_g_loss(self, det_g: torch.Tensor) -> torch.Tensor:
        """Loss for matching metric determinant to 65/32."""
        target = self.zpg.det_g_target
        mean_det = det_g.mean()
        var_det = det_g.var()
        return self.weights['det_g'] * ((mean_det - target)**2 + 0.1 * var_det)
    
    def closure_loss(self, phi: torch.Tensor, x: torch.Tensor, model: nn.Module) -> torch.Tensor:
        """Loss for d(phi) = 0 (closure condition)."""
        eps = 1e-4
        d_phi_norm_sq = torch.zeros(phi.shape[0], device=phi.device, dtype=phi.dtype)
        
        for dim in range(7):
            x_plus = x.clone()
            x_plus[:, dim] += eps
            x_minus = x.clone() 
            x_minus[:, dim] -= eps
            
            with torch.no_grad():
                out_plus = model(x_plus)
                out_minus = model(x_minus)
            
            dphi = (out_plus['phi_total'] - out_minus['phi_total']) / (2 * eps)
            d_phi_norm_sq += (dphi**2).sum(dim=(-1, -2, -3))
        
        return self.weights['closure'] * d_phi_norm_sq.mean()
    
    def coclosure_loss(self, phi: torch.Tensor, g: torch.Tensor) -> torch.Tensor:
        """Loss for d*(phi) = 0 (coclosure condition)."""
        g_inv = torch.linalg.inv(g)
        contracted = torch.einsum('bij,bjkl->bikl', g_inv, phi)
        coclosure_measure = (contracted**2).sum(dim=(-1, -2, -3))
        return self.weights['coclosure'] * coclosure_measure.mean()
    
    def g2_consistency_loss(self, phi: torch.Tensor) -> torch.Tensor:
        """Loss for G2 structure preservation."""
        phi_norm_sq = (phi**2).sum(dim=(-1, -2, -3))
        target_norm_sq = 7.0 * 6.0
        consistency = (phi_norm_sq - target_norm_sq)**2
        return self.weights['g2_consistency'] * consistency.mean()
    
    def local_global_balance_loss(self, phi_local: torch.Tensor, 
                                   phi_global: torch.Tensor) -> torch.Tensor:
        """Regularize balance between local and global components."""
        local_norm = (phi_local**2).sum(dim=(-1, -2, -3)).mean()
        global_norm = (phi_global**2).sum(dim=(-1, -2, -3)).mean()
        
        ratio = global_norm / (local_norm + 1e-12)
        target_ratio = 0.2
        balance_loss = (ratio - target_ratio)**2
        global_activity = -torch.log(global_norm + 1e-12) * 0.01
        
        return self.weights['local_global_balance'] * (balance_loss + global_activity)
    
    def spd_loss(self, g: torch.Tensor) -> torch.Tensor:
        """Loss to enforce positive definiteness of metric."""
        eigenvalues = torch.linalg.eigvalsh(g)
        min_eigenvalues = eigenvalues.min(dim=-1)[0]
        negative_penalty = F.relu(-min_eigenvalues + 0.01)**2
        return self.weights['spd'] * negative_penalty.mean()
    
    def total_loss(self, model_output: Dict[str, torch.Tensor], 
                   x: torch.Tensor, model: nn.Module,
                   phase: str = 'both') -> Tuple[torch.Tensor, Dict[str, float]]:
        """
        Compute total loss with all components.
        
        v1.5b phases:
        - 'global_only': Train only global, local frozen
        - 'both': Train both with anchor losses
        """
        losses = {}
        
        # Core losses (always active)
        losses['kappa_T'] = self.kappa_T_loss(model_output['torsion'])
        losses['det_g'] = self.det_g_loss(model_output['det_g'])
        losses['g2_consistency'] = self.g2_consistency_loss(model_output['phi_total'])
        losses['spd'] = self.spd_loss(model_output['g'])
        
        # v1.5b: Separated torsion losses
        losses['local_anchor'] = self.local_anchor_loss(
            model_output['phi_local'], x)
        losses['global_torsion'] = self.global_torsion_loss(
            model_output['phi_global'], x)
        
        # Mode activation (encourage all 42 modes)
        if 'c' in model_output:
            losses['mode_activation'] = self.mode_activation_loss(model_output['c'])
        
        # Phase-dependent losses
        if phase in ['both']:
            losses['closure'] = self.closure_loss(
                model_output['phi_total'], x, model) * 0.1
            losses['local_global_balance'] = self.local_global_balance_loss(
                model_output['phi_local'], model_output['phi_global'])
        
        # Total
        total = sum(losses.values())
        
        # Convert to float dict for logging
        loss_dict = {k: v.item() for k, v in losses.items()}
        loss_dict['total'] = total.item()
        
        return total, loss_dict


# Initialize loss functions
loss_fn = GIFTLossFunctions(ZPG, SC, CONFIG)
print("Loss functions initialized (v1.5b with torsion separation)")
print(f"  kappa_T reference: {CONFIG.get('v14_kappa_T_ref', ZPG.kappa_T):.6f}")
print(f"  Weights:")
for k, v in CONFIG['loss_weights'].items():
    print(f"    {k}: {v}")

# Test loss computation
test_loss, test_loss_dict = loss_fn.total_loss(test_out, test_x, model, phase='both')
print(f"\nTest loss computation:")
for name, val in test_loss_dict.items():
    print(f"  {name}: {val:.6f}")

## 11. Training Loop

Multi-phase training schedule:
1. **Warmup** (50 epochs): Train local network only
2. **Local stabilize** (150 epochs): Continue local training to achieve kappa_T, det_g
3. **Global activate** (150 epochs): Train both networks, activate global modes
4. **Fine tune** (150 epochs): Joint fine-tuning for b3 = 77

In [ ]:
def load_v14_weights(model: CombinedG2Model, config: Dict) -> bool:
    """
    Attempt to load v1.4 weights for local network initialization.
    Returns True if successful, False otherwise.
    """
    v14_path = config.get('v14_model_path')
    if not v14_path or not os.path.exists(v14_path):
        print(f"  [INFO] v1.4 model not found at {v14_path}, starting fresh")
        return False
    
    try:
        checkpoint = torch.load(v14_path, map_location=device)
        # v1.4 might have different structure - try to load what we can
        if isinstance(checkpoint, dict):
            print(f"  [INFO] v1.4 checkpoint has keys: {list(checkpoint.keys())}")
            # Try loading state dict if available
            # Note: v1.4 architecture may differ, so we do best-effort loading
        print(f"  [OK] Loaded v1.4 checkpoint from {v14_path}")
        return True
    except Exception as e:
        print(f"  [WARN] Could not load v1.4 weights: {e}")
        return False


def freeze_network(net: nn.Module, freeze: bool = True):
    """Freeze or unfreeze all parameters in a network."""
    for param in net.parameters():
        param.requires_grad = not freeze


def train_model_v15b(model: CombinedG2Model, loss_fn: GIFTLossFunctions, 
                     config: Dict, zpg: ZeroParamGeometry) -> Dict:
    """
    v1.5b Multi-phase training loop with freeze/unfreeze strategy.
    
    Key improvements:
    - Freeze local network in early phases (protect v1.4 kappa_T)
    - Gradually unfreeze with low LR and anchor loss
    - Track separated torsion metrics
    
    Returns:
        history: Dictionary with training metrics per epoch
    """
    print(f"\n{'='*60}")
    print("GIFT K7 v1.5b TRAINING")
    print(f"{'='*60}")
    
    # Try to load v1.4 weights
    v14_loaded = load_v14_weights(model, config)
    if v14_loaded:
        print("  Starting from v1.4 solution (local network anchored)")
    else:
        print("  Starting fresh (no v1.4 anchor)")
    
    # Optimizers (separate for local and global)
    opt_local = Adam(model.local_net.parameters(), 
                     lr=config['lr_local'], weight_decay=config['weight_decay'])
    opt_global = Adam(model.global_net.parameters(),
                      lr=config['lr_global'], weight_decay=config['weight_decay'])
    
    # Learning rate schedulers
    total_epochs = sum(p['epochs'] for p in config['phases'])
    scheduler_local = CosineAnnealingLR(opt_local, T_max=total_epochs, eta_min=1e-7)
    scheduler_global = CosineAnnealingLR(opt_global, T_max=total_epochs, eta_min=1e-6)
    
    # Training history
    history = {
        'epoch': [], 'phase': [], 'loss_total': [], 
        'kappa_T': [], 'det_g': [], 
        'local_norm': [], 'global_norm': [],
        'T_local': [], 'T_global': [],  # v1.5b: separated torsion tracking
    }
    
    epoch = 0
    
    for phase_info in config['phases']:
        phase_name = phase_info['name']
        phase_epochs = phase_info['epochs']
        phase_focus = phase_info.get('focus', 'both')
        freeze_local = phase_info.get('freeze_local', False)
        local_lr_factor = phase_info.get('local_lr_factor', 1.0)
        
        print(f"\n--- Phase: {phase_name} ---")
        print(f"    Epochs: {phase_epochs}, Focus: {phase_focus}")
        print(f"    Local frozen: {freeze_local}, Local LR factor: {local_lr_factor}")
        
        # Apply freeze setting
        freeze_network(model.local_net, freeze_local)
        
        # Adjust local LR if not frozen
        if not freeze_local and local_lr_factor != 1.0:
            for pg in opt_local.param_groups:
                pg['lr'] = config['lr_local'] * local_lr_factor
            print(f"    Local LR adjusted to: {config['lr_local'] * local_lr_factor:.2e}")
        
        for ep in range(phase_epochs):
            # Sample random coordinates
            x = torch.rand(config['n_points'], 7, device=device, dtype=torch.float64)
            
            # Zero gradients
            opt_local.zero_grad()
            opt_global.zero_grad()
            
            # Forward pass
            out = model(x)
            
            # Compute loss based on phase
            loss, loss_dict = loss_fn.total_loss(out, x, model, phase=phase_focus)
            
            # Backward pass
            loss.backward()
            
            # Update parameters based on phase
            if not freeze_local and phase_focus in ['local', 'both']:
                torch.nn.utils.clip_grad_norm_(model.local_net.parameters(), 1.0)
                opt_local.step()
            
            if phase_focus in ['global', 'global_only', 'both']:
                torch.nn.utils.clip_grad_norm_(model.global_net.parameters(), 1.0)
                opt_global.step()
            
            # Step schedulers (only if not frozen)
            if not freeze_local:
                scheduler_local.step()
            scheduler_global.step()
            
            # Compute metrics
            with torch.no_grad():
                local_norm = (out['phi_local']**2).sum(dim=(-1,-2,-3)).mean().item()
                global_norm = (out['phi_global']**2).sum(dim=(-1,-2,-3)).mean().item()
                mean_torsion = out['torsion'].mean().item()
                mean_det_g = out['det_g'].mean().item()
                
                # v1.5b: Separated torsion
                T_local = loss_fn.compute_torsion_norm(out['phi_local'], x).mean().item()
                T_global = loss_fn.compute_torsion_norm(out['phi_global'], x).mean().item()
            
            # Record history
            history['epoch'].append(epoch)
            history['phase'].append(phase_name)
            history['loss_total'].append(loss_dict['total'])
            history['kappa_T'].append(mean_torsion)
            history['det_g'].append(mean_det_g)
            history['local_norm'].append(local_norm)
            history['global_norm'].append(global_norm)
            history['T_local'].append(T_local)
            history['T_global'].append(T_global)
            
            # Print progress
            if ep % 50 == 0 or ep == phase_epochs - 1:
                print(f"  Ep {epoch:4d} | Loss: {loss_dict['total']:.3f} | "
                      f"kT: {mean_torsion:.4f} | det: {mean_det_g:.4f} | "
                      f"T_loc: {T_local:.4f} | T_glob: {T_global:.4f}")
            
            epoch += 1
    
    # Unfreeze local at end
    freeze_network(model.local_net, False)
    
    print(f"\n{'='*60}")
    print("TRAINING COMPLETE")
    print(f"{'='*60}")
    
    # Final metrics
    print(f"\nFinal Results:")
    print(f"  kappa_T achieved: {history['kappa_T'][-1]:.6f} (target: {zpg.kappa_T:.6f})")
    print(f"  det_g achieved: {history['det_g'][-1]:.6f} (target: {zpg.det_g_target:.6f})")
    print(f"  T_local: {history['T_local'][-1]:.6f}")
    print(f"  T_global: {history['T_global'][-1]:.6f}")
    print(f"  Local phi norm: {history['local_norm'][-1]:.4f}")
    print(f"  Global phi norm: {history['global_norm'][-1]:.4f}")
    
    # Deviation report
    kappa_dev = abs(history['kappa_T'][-1] - zpg.kappa_T) / zpg.kappa_T * 100
    det_dev = abs(history['det_g'][-1] - zpg.det_g_target) / zpg.det_g_target * 100
    print(f"\nDeviations:")
    print(f"  kappa_T: {kappa_dev:.2f}%")
    print(f"  det_g: {det_dev:.2f}%")
    
    return history


# Run training
TRAIN = True  # Set to False to skip training

if TRAIN:
    history = train_model_v15b(model, loss_fn, CONFIG, ZPG)
else:
    print("Skipping training (TRAIN=False)")
    history = None

## 12. Harmonic Extraction (Betti Numbers)

Compute effective Betti numbers via Gram matrix eigenvalues:
- b2_eff from 2-form basis
- b3_eff_local from local 3-form basis (35)
- b3_eff_global from global 3-form basis (42)
- b3_eff_total from combined basis (77)

In [ ]:
class BettiNumberExtractor:
    """
    Extract effective Betti numbers via Gram matrix eigenvalue analysis.
    
    For a basis {omega_i} of forms, the Gram matrix is:
        G_ij = <omega_i, omega_j> = integral over K7 of omega_i ^ *omega_j
    
    The effective dimension (Betti number) is the number of significant eigenvalues.
    """
    
    def __init__(self, model: CombinedG2Model, local_basis: LocalG2Basis,
                 global_basis: GlobalBasis, sc: StructuralConstants, config: Dict):
        self.model = model
        self.local_basis = local_basis
        self.global_basis = global_basis
        self.sc = sc
        self.threshold = config['betti_threshold']
        self.n_samples = config['n_betti_samples']
    
    @torch.no_grad()
    def compute_gram_matrix(self, forms: List[torch.Tensor], 
                           g: torch.Tensor) -> torch.Tensor:
        """
        Compute Gram matrix for a set of forms using the metric g.
        
        Args:
            forms: List of tensors, each shape (7, 7, 7) for 3-forms
            g: Metric tensor (batch, 7, 7) or (7, 7)
            
        Returns:
            Gram matrix of shape (n_forms, n_forms)
        """
        n_forms = len(forms)
        gram = torch.zeros(n_forms, n_forms, device=device, dtype=torch.float64)
        
        # Use average metric if batched
        if g.dim() == 3:
            g_avg = g.mean(dim=0)
        else:
            g_avg = g
        
        # Compute det(g) for volume element
        det_g = torch.linalg.det(g_avg)
        vol_factor = torch.sqrt(det_g.abs() + 1e-12)
        
        # Inverse metric for raising indices
        g_inv = torch.linalg.inv(g_avg)
        
        for i in range(n_forms):
            for j in range(i, n_forms):
                # Inner product: <omega_i, omega_j> = g^{abc} g^{def} omega_i_{acd} omega_j_{bef}
                # Simplified: flat metric inner product scaled by vol_factor
                inner = (forms[i] * forms[j]).sum() * vol_factor
                gram[i, j] = inner
                gram[j, i] = inner
        
        return gram
    
    @torch.no_grad()
    def extract_b2(self) -> Dict[str, float]:
        """
        Extract b2_eff from 2-form basis.
        
        For b2, we use the harmonic 2-forms derived from the K7 metric.
        In this simplified model, we use the Jacobi matrix of phi.
        """
        # Sample points
        x = torch.rand(self.n_samples, 7, device=device, dtype=torch.float64)
        out = self.model(x)
        g = out['g']
        
        # Build 2-form basis from metric derivatives
        # The 2-forms are approximately the curvature 2-forms
        # For simplicity, use C(7,2) = 21 coordinate 2-forms
        forms_2 = []
        for i in range(7):
            for j in range(i+1, 7):
                omega_2 = torch.zeros(7, 7, device=device, dtype=torch.float64)
                omega_2[i, j] = 1.0
                omega_2[j, i] = -1.0
                forms_2.append(omega_2)
        
        # Compute Gram matrix for 2-forms
        n_2forms = len(forms_2)
        gram_2 = torch.zeros(n_2forms, n_2forms, device=device, dtype=torch.float64)
        
        g_avg = g.mean(dim=0)
        g_inv = torch.linalg.inv(g_avg)
        det_g = torch.linalg.det(g_avg)
        vol_factor = torch.sqrt(det_g.abs() + 1e-12)
        
        for i in range(n_2forms):
            for j in range(i, n_2forms):
                # <omega_i, omega_j> = g^{ac} g^{bd} omega_i_{ab} omega_j_{cd} * sqrt(det g)
                inner = torch.einsum('ac,bd,ab,cd->', g_inv, g_inv, 
                                    forms_2[i], forms_2[j]) * vol_factor
                gram_2[i, j] = inner
                gram_2[j, i] = inner
        
        # Eigenvalue analysis
        eigenvalues = torch.linalg.eigvalsh(gram_2)
        eigenvalues = eigenvalues.sort(descending=True)[0]
        
        # Count significant eigenvalues
        max_eig = eigenvalues[0].abs()
        threshold = self.threshold * max_eig
        b2_eff = (eigenvalues.abs() > threshold).sum().item()
        
        return {
            'b2_eff': b2_eff,
            'b2_target': self.sc.b2_K7,
            'b2_match': abs(b2_eff - self.sc.b2_K7) <= 1,
            'eigenvalues_2form': eigenvalues.cpu().numpy()[:5],  # Top 5
        }
    
    @torch.no_grad()
    def extract_b3_local(self) -> Dict[str, float]:
        """Extract b3_eff from local 35-dimensional basis."""
        # Sample points
        x = torch.rand(self.n_samples, 7, device=device, dtype=torch.float64)
        out = self.model(x)
        g = out['g']
        
        # Compute Gram matrix for local basis (35 forms)
        gram_local = self.compute_gram_matrix(self.local_basis.local_basis, g)
        
        # Eigenvalue analysis
        eigenvalues = torch.linalg.eigvalsh(gram_local)
        eigenvalues = eigenvalues.sort(descending=True)[0]
        
        max_eig = eigenvalues[0].abs()
        threshold = self.threshold * max_eig
        b3_local_eff = (eigenvalues.abs() > threshold).sum().item()
        
        return {
            'b3_local_eff': b3_local_eff,
            'b3_local_target': self.sc.local_dim,  # 35
            'b3_local_match': abs(b3_local_eff - self.sc.local_dim) <= 2,
            'eigenvalues_local': eigenvalues.cpu().numpy()[:5],
        }
    
    @torch.no_grad()
    def extract_b3_global(self) -> Dict[str, float]:
        """Extract b3_eff from global 42-dimensional basis."""
        # Sample points
        x = torch.rand(self.n_samples, 7, device=device, dtype=torch.float64)
        out = self.model(x)
        g = out['g']
        
        # Compute Gram matrix for global basis (42 forms)
        gram_global = self.compute_gram_matrix(self.global_basis.global_basis, g)
        
        # Eigenvalue analysis
        eigenvalues = torch.linalg.eigvalsh(gram_global)
        eigenvalues = eigenvalues.sort(descending=True)[0]
        
        max_eig = eigenvalues[0].abs()
        threshold = self.threshold * max_eig
        b3_global_eff = (eigenvalues.abs() > threshold).sum().item()
        
        return {
            'b3_global_eff': b3_global_eff,
            'b3_global_target': self.sc.global_dim,  # 42
            'b3_global_match': abs(b3_global_eff - self.sc.global_dim) <= 2,
            'eigenvalues_global': eigenvalues.cpu().numpy()[:5],
        }
    
    @torch.no_grad()
    def extract_b3_total(self) -> Dict[str, float]:
        """Extract b3_eff from combined 77-dimensional basis."""
        # Sample points
        x = torch.rand(self.n_samples, 7, device=device, dtype=torch.float64)
        out = self.model(x)
        g = out['g']
        
        # Combined basis (local + global = 35 + 42 = 77)
        combined_basis = self.local_basis.local_basis + self.global_basis.global_basis
        
        # Compute Gram matrix for combined basis
        gram_combined = self.compute_gram_matrix(combined_basis, g)
        
        # Eigenvalue analysis
        eigenvalues = torch.linalg.eigvalsh(gram_combined)
        eigenvalues = eigenvalues.sort(descending=True)[0]
        
        max_eig = eigenvalues[0].abs()
        threshold = self.threshold * max_eig
        b3_total_eff = (eigenvalues.abs() > threshold).sum().item()
        
        return {
            'b3_total_eff': b3_total_eff,
            'b3_total_target': self.sc.b3_K7,  # 77
            'b3_total_match': abs(b3_total_eff - self.sc.b3_K7) <= 3,
            'eigenvalues_combined': eigenvalues.cpu().numpy()[:10],
        }
    
    def full_extraction(self) -> Dict[str, any]:
        """Run full Betti number extraction."""
        print("\n" + "="*60)
        print("BETTI NUMBER EXTRACTION")
        print("="*60)
        
        results = {}
        
        # Extract b2
        print("\nExtracting b2 from 2-form basis...")
        b2_results = self.extract_b2()
        results.update(b2_results)
        status_b2 = "OK" if b2_results['b2_match'] else "MISMATCH"
        print(f"  b2_eff = {b2_results['b2_eff']} (target: {b2_results['b2_target']}) [{status_b2}]")
        
        # Extract b3 local
        print("\nExtracting b3_local from 35-dim local basis...")
        b3_local_results = self.extract_b3_local()
        results.update(b3_local_results)
        status_local = "OK" if b3_local_results['b3_local_match'] else "MISMATCH"
        print(f"  b3_local_eff = {b3_local_results['b3_local_eff']} (target: {b3_local_results['b3_local_target']}) [{status_local}]")
        
        # Extract b3 global
        print("\nExtracting b3_global from 42-dim global basis...")
        b3_global_results = self.extract_b3_global()
        results.update(b3_global_results)
        status_global = "OK" if b3_global_results['b3_global_match'] else "MISMATCH"
        print(f"  b3_global_eff = {b3_global_results['b3_global_eff']} (target: {b3_global_results['b3_global_target']}) [{status_global}]")
        
        # Extract b3 total
        print("\nExtracting b3_total from 77-dim combined basis...")
        b3_total_results = self.extract_b3_total()
        results.update(b3_total_results)
        status_total = "OK" if b3_total_results['b3_total_match'] else "MISMATCH"
        print(f"  b3_total_eff = {b3_total_results['b3_total_eff']} (target: {b3_total_results['b3_total_target']}) [{status_total}]")
        
        # Summary
        print("\n" + "-"*60)
        print("SUMMARY:")
        print(f"  b2:       {results['b2_eff']:3d} / {self.sc.b2_K7} (2-forms)")
        print(f"  b3_local: {results['b3_local_eff']:3d} / {self.sc.local_dim} (local 3-forms)")
        print(f"  b3_global:{results['b3_global_eff']:3d} / {self.sc.global_dim} (global 3-forms)")
        print(f"  b3_total: {results['b3_total_eff']:3d} / {self.sc.b3_K7} (combined)")
        print("="*60)
        
        return results


# Run Betti number extraction
print("Initializing Betti number extractor...")
betti_extractor = BettiNumberExtractor(model, LOCAL_BASIS, GLOBAL_BASIS, SC, CONFIG)

if TRAIN or True:  # Always run extraction
    betti_results = betti_extractor.full_extraction()
else:
    betti_results = None

## 13. Representation Diagnostics ((2, 21, 54) Decomposition)

Verify that H3(K7) = 77 decomposes as:
- n1 = 2 singlets (1 local + 1 global)
- n7 = 3 copies of 7-rep (1 local + 2 global) = 21 dimensions
- n27 = 2 copies of 27-rep (1 local + 1 global) = 54 dimensions

Total: 2 + 21 + 54 = 77

In [ ]:
class RepresentationDiagnostics:
    """
    Diagnostics for the (2, 21, 54) G2 representation decomposition of H3(K7).
    
    The 77-dimensional H3(K7) decomposes under G2 as:
    - 2 copies of the singlet (1-dim)
    - 3 copies of the fundamental (7-dim)
    - 2 copies of the symmetric traceless (27-dim)
    
    Local contribution: (1, 7, 27) from Lambda^3 decomposition
    Global contribution: (1, 14, 27) from TCS topology
    """
    
    def __init__(self, model: CombinedG2Model, local_basis: LocalG2Basis,
                 global_basis: GlobalBasis, sc: StructuralConstants):
        self.model = model
        self.local_basis = local_basis
        self.global_basis = global_basis
        self.sc = sc
    
    def compute_casimir(self, forms: List[torch.Tensor]) -> torch.Tensor:
        """
        Compute quadratic Casimir eigenvalues to identify G2 representations.
        
        Casimir eigenvalues:
        - Singlet (1): C2 = 0
        - Fundamental (7): C2 = 4
        - Adjoint (14): C2 = 4
        - 27-rep: C2 = 6
        """
        n_forms = len(forms)
        if n_forms == 0:
            return torch.tensor([])
        
        # Build representation matrix via G2 structure constants
        # Simplified: use phi contraction as a proxy
        phi = self.local_basis.phi_canonical
        
        casimir_values = []
        for form in forms:
            # Contract with phi to get Casimir-like measure
            # For singlet (proportional to phi), this should be maximal
            contracted = torch.einsum('ijk,ijk->', form, phi)
            form_norm = torch.sqrt((form**2).sum() + 1e-12)
            phi_norm = torch.sqrt((phi**2).sum())
            
            # Normalized overlap with phi (1 for singlet, <1 for others)
            overlap = contracted.abs() / (form_norm * phi_norm + 1e-12)
            casimir_values.append(overlap.item())
        
        return torch.tensor(casimir_values)
    
    @torch.no_grad()
    def analyze_local(self) -> Dict[str, any]:
        """Analyze local representation content (should be 1+7+27=35)."""
        results = {
            'n_singlet_local': len(self.local_basis.basis_1),
            'n_7rep_local': len(self.local_basis.basis_7),
            'n_27rep_local': len(self.local_basis.basis_27),
            'total_local': self.local_basis.get_local_dim(),
        }
        
        # Verify dimensions match Lambda^3 decomposition
        results['singlet_dim_ok'] = results['n_singlet_local'] == self.sc.dim_Lambda3_1
        results['7rep_dim_ok'] = results['n_7rep_local'] == self.sc.dim_Lambda3_7
        results['27rep_dim_ok'] = results['n_27rep_local'] == self.sc.dim_Lambda3_27
        
        # Casimir analysis
        casimir_1 = self.compute_casimir(self.local_basis.basis_1)
        casimir_7 = self.compute_casimir(self.local_basis.basis_7)
        casimir_27 = self.compute_casimir(self.local_basis.basis_27)
        
        results['casimir_singlet_mean'] = casimir_1.mean().item() if len(casimir_1) > 0 else 0
        results['casimir_7rep_mean'] = casimir_7.mean().item() if len(casimir_7) > 0 else 0
        results['casimir_27rep_mean'] = casimir_27.mean().item() if len(casimir_27) > 0 else 0
        
        return results
    
    @torch.no_grad()
    def analyze_global(self) -> Dict[str, any]:
        """Analyze global representation content (should be 1+14+27=42)."""
        results = {
            'n_singlet_global': self.global_basis.n_extra_singlet,  # 1
            'n_7rep_global': self.global_basis.n_extra_7rep,        # 14 (2 copies of 7)
            'n_27rep_global': self.global_basis.n_extra_27rep,      # 27
            'total_global': self.global_basis.get_global_dim(),
        }
        
        # Verify dimensions
        results['singlet_ok'] = results['n_singlet_global'] == 1
        results['7rep_ok'] = results['n_7rep_global'] == 14
        results['27rep_ok'] = results['n_27rep_global'] == 27
        results['total_ok'] = results['total_global'] == self.sc.global_dim
        
        return results
    
    @torch.no_grad()
    def analyze_combined(self) -> Dict[str, any]:
        """Analyze total (2, 21, 54) decomposition."""
        local_res = self.analyze_local()
        global_res = self.analyze_global()
        
        # Total counts
        n1_total = local_res['n_singlet_local'] + global_res['n_singlet_global']
        n7_total = local_res['n_7rep_local'] + global_res['n_7rep_global']
        n27_total = local_res['n_27rep_local'] + global_res['n_27rep_global']
        
        results = {
            'n1_total': n1_total,  # Should be 2
            'n7_total_dims': n7_total,  # Should be 21 (7 + 14)
            'n27_total_dims': n27_total,  # Should be 54 (27 + 27)
            'grand_total': n1_total + n7_total + n27_total,  # Should be 77
            
            # Multiplicity counts (n_rep, not dims)
            'n1_multiplicity': n1_total,  # 2 singlets
            'n7_multiplicity': n7_total // 7 if n7_total >= 7 else 0,  # 3 copies of 7
            'n27_multiplicity': n27_total // 27 if n27_total >= 27 else 0,  # 2 copies of 27
        }
        
        # Verify (2, 21, 54) pattern
        results['matches_2_21_54'] = (
            results['n1_total'] == self.sc.n_singlets_global and
            results['n7_total_dims'] == self.sc.n_7rep_global * self.sc.dim_Lambda3_7 and
            results['n27_total_dims'] == self.sc.n_27rep_global * self.sc.dim_Lambda3_27
        )
        
        return results, local_res, global_res
    
    def full_diagnostics(self) -> Dict[str, any]:
        """Run full representation diagnostics."""
        print("\n" + "="*60)
        print("REPRESENTATION DIAGNOSTICS: (2, 21, 54)")
        print("="*60)
        
        combined, local_res, global_res = self.analyze_combined()
        
        # Local analysis
        print("\nLOCAL DECOMPOSITION (Lambda^3 = 1 + 7 + 27 = 35):")
        print(f"  Lambda^3_1 (singlet):   {local_res['n_singlet_local']} forms")
        print(f"  Lambda^3_7 (fund):      {local_res['n_7rep_local']} forms")
        print(f"  Lambda^3_27 (sym-tr):   {local_res['n_27rep_local']} forms")
        print(f"  Total local:            {local_res['total_local']} forms")
        
        # Global analysis
        print("\nGLOBAL DECOMPOSITION (TCS = 1 + 14 + 27 = 42):")
        print(f"  Extra singlet:          {global_res['n_singlet_global']} forms")
        print(f"  Extra 7-rep (2x7):      {global_res['n_7rep_global']} forms")
        print(f"  Extra 27-rep:           {global_res['n_27rep_global']} forms")
        print(f"  Total global:           {global_res['total_global']} forms")
        
        # Combined (2, 21, 54) verification
        print("\nCOMBINED (2, 21, 54) DECOMPOSITION:")
        print(f"  Singlets (n1=2):        {combined['n1_total']} dims  [target: 2]")
        print(f"  7-reps (n7=3 -> 21):    {combined['n7_total_dims']} dims  [target: 21]")
        print(f"  27-reps (n27=2 -> 54):  {combined['n27_total_dims']} dims  [target: 54]")
        print(f"  Grand total:            {combined['grand_total']} dims  [target: 77]")
        
        status = "OK" if combined['matches_2_21_54'] else "MISMATCH"
        print(f"\n  (2, 21, 54) pattern: [{status}]")
        
        # Multiplicity counts
        print("\nMULTIPLICITY SUMMARY:")
        print(f"  n1 (singlet count):     {combined['n1_multiplicity']} copies")
        print(f"  n7 (7-rep count):       {combined['n7_multiplicity']} copies")
        print(f"  n27 (27-rep count):     {combined['n27_multiplicity']} copies")
        
        print("="*60)
        
        return {
            'combined': combined,
            'local': local_res,
            'global': global_res,
        }


# Run representation diagnostics
rep_diag = RepresentationDiagnostics(model, LOCAL_BASIS, GLOBAL_BASIS, SC)
rep_results = rep_diag.full_diagnostics()

## 14. Output and Save

Save trained models, results, and metadata:
- `models_v1_5.pt`: Neural network weights
- `results_v1_5.json`: All numerical results
- `results_v1_5.tex`: LaTeX table for publication
- Training history and configuration

In [ ]:
def save_results(model: CombinedG2Model, history: Dict, betti_results: Dict,
                 rep_results: Dict, zpg: ZeroParamGeometry, sc: StructuralConstants,
                 config: Dict, output_dir: str = '.'):
    """Save all results to files."""
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    print("\n" + "="*60)
    print("SAVING RESULTS")
    print("="*60)
    
    # 1. Save model weights
    model_path = os.path.join(output_dir, 'models_v1_5.pt')
    torch.save({
        'local_net_state_dict': model.local_net.state_dict(),
        'global_net_state_dict': model.global_net.state_dict(),
        'config': config,
        'timestamp': timestamp,
    }, model_path)
    print(f"  Model weights: {model_path}")
    
    # 2. Sample coordinates and compute final metrics
    x_sample = torch.rand(1024, 7, device=device, dtype=torch.float64)
    with torch.no_grad():
        out = model(x_sample)
        final_kappa_T = out['torsion'].mean().item()
        final_det_g = out['det_g'].mean().item()
        local_norm = (out['phi_local']**2).sum(dim=(-1,-2,-3)).mean().item()
        global_norm = (out['phi_global']**2).sum(dim=(-1,-2,-3)).mean().item()
    
    # 3. Compile results dictionary
    results = {
        'version': '1.5',
        'timestamp': timestamp,
        'targets': {
            'kappa_T': str(zpg.kappa_T_fraction),
            'det_g': str(zpg.det_g_fraction),
            'b2': sc.b2_K7,
            'b3': sc.b3_K7,
            'b3_local': sc.local_dim,
            'b3_global': sc.global_dim,
        },
        'achieved': {
            'kappa_T': final_kappa_T,
            'det_g': final_det_g,
            'local_phi_norm': local_norm,
            'global_phi_norm': global_norm,
        },
        'betti_numbers': {
            'b2_eff': betti_results['b2_eff'] if betti_results else None,
            'b3_local_eff': betti_results['b3_local_eff'] if betti_results else None,
            'b3_global_eff': betti_results['b3_global_eff'] if betti_results else None,
            'b3_total_eff': betti_results['b3_total_eff'] if betti_results else None,
        },
        'representation': {
            'n1': rep_results['combined']['n1_total'],
            'n7_dims': rep_results['combined']['n7_total_dims'],
            'n27_dims': rep_results['combined']['n27_total_dims'],
            'matches_2_21_54': rep_results['combined']['matches_2_21_54'],
        },
        'deviations': {
            'kappa_T_rel': abs(final_kappa_T - zpg.kappa_T) / zpg.kappa_T * 100,
            'det_g_rel': abs(final_det_g - zpg.det_g_target) / zpg.det_g_target * 100,
        },
        'training': {
            'n_epochs': len(history['epoch']) if history else 0,
            'final_loss': history['loss_total'][-1] if history else None,
        },
        'config': config,
    }
    
    # Save JSON
    json_path = os.path.join(output_dir, 'results_v1_5.json')
    
    # Convert non-serializable items
    def make_serializable(obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, (np.float32, np.float64)):
            return float(obj)
        if isinstance(obj, (np.int32, np.int64)):
            return int(obj)
        if isinstance(obj, dict):
            return {k: make_serializable(v) for k, v in obj.items()}
        if isinstance(obj, list):
            return [make_serializable(i) for i in obj]
        return obj
    
    with open(json_path, 'w') as f:
        json.dump(make_serializable(results), f, indent=2)
    print(f"  Results JSON: {json_path}")
    
    # 4. Generate LaTeX table
    tex_content = r"""\begin{table}[h]
\centering
\caption{GIFT K7 v1.5 Results: Local/Global G2 Decomposition}
\label{tab:gift_v1_5}
\begin{tabular}{lrrr}
\toprule
\textbf{Observable} & \textbf{Target} & \textbf{Achieved} & \textbf{Deviation} \\
\midrule
$\kappa_T$ (torsion) & $1/61$ & """ + f"{final_kappa_T:.6f}" + r""" & """ + f"{results['deviations']['kappa_T_rel']:.2f}" + r"""\% \\
$\det(g)$ (metric det) & $65/32$ & """ + f"{final_det_g:.6f}" + r""" & """ + f"{results['deviations']['det_g_rel']:.2f}" + r"""\% \\
\midrule
$b_2$ (2-forms) & 21 & """ + f"{betti_results['b2_eff'] if betti_results else 'N/A'}" + r""" & -- \\
$b_3^{\text{local}}$ & 35 & """ + f"{betti_results['b3_local_eff'] if betti_results else 'N/A'}" + r""" & -- \\
$b_3^{\text{global}}$ & 42 & """ + f"{betti_results['b3_global_eff'] if betti_results else 'N/A'}" + r""" & -- \\
$b_3^{\text{total}}$ & 77 & """ + f"{betti_results['b3_total_eff'] if betti_results else 'N/A'}" + r""" & -- \\
\midrule
$(n_1, n_7 \cdot 7, n_{27} \cdot 27)$ & (2, 21, 54) & """ + f"({rep_results['combined']['n1_total']}, {rep_results['combined']['n7_total_dims']}, {rep_results['combined']['n27_total_dims']})" + r""" & -- \\
\bottomrule
\end{tabular}
\end{table}
"""
    
    tex_path = os.path.join(output_dir, 'results_v1_5.tex')
    with open(tex_path, 'w') as f:
        f.write(tex_content)
    print(f"  LaTeX table: {tex_path}")
    
    # 5. Save sample coordinates and outputs
    coords_path = os.path.join(output_dir, 'sample_coords_v1_5.pt')
    torch.save({
        'x': x_sample.cpu(),
        'phi_local': out['phi_local'].cpu(),
        'phi_global': out['phi_global'].cpu(),
        'phi_total': out['phi_total'].cpu(),
        'g': out['g'].cpu(),
        'det_g': out['det_g'].cpu(),
        'torsion': out['torsion'].cpu(),
    }, coords_path)
    print(f"  Sample coordinates: {coords_path}")
    
    # 6. Save training history
    if history:
        hist_path = os.path.join(output_dir, 'history_v1_5.json')
        with open(hist_path, 'w') as f:
            json.dump(make_serializable(history), f, indent=2)
        print(f"  Training history: {hist_path}")
    
    print("\n" + "="*60)
    print("FINAL SUMMARY")
    print("="*60)
    print(f"\n  kappa_T: {final_kappa_T:.6f} (target: {zpg.kappa_T:.6f}, dev: {results['deviations']['kappa_T_rel']:.2f}%)")
    print(f"  det(g):  {final_det_g:.6f} (target: {zpg.det_g_target:.6f}, dev: {results['deviations']['det_g_rel']:.2f}%)")
    print(f"\n  Betti numbers: b2={betti_results['b2_eff'] if betti_results else 'N/A'}, " +
          f"b3_local={betti_results['b3_local_eff'] if betti_results else 'N/A'}, " +
          f"b3_global={betti_results['b3_global_eff'] if betti_results else 'N/A'}, " +
          f"b3_total={betti_results['b3_total_eff'] if betti_results else 'N/A'}")
    print(f"\n  Representation: (2, 21, 54) = ({rep_results['combined']['n1_total']}, " +
          f"{rep_results['combined']['n7_total_dims']}, {rep_results['combined']['n27_total_dims']})")
    
    match_status = "MATCH" if rep_results['combined']['matches_2_21_54'] else "MISMATCH"
    print(f"  Status: {match_status}")
    print("="*60)
    
    return results


# Save results
if history is not None or True:
    final_results = save_results(model, history, betti_results, rep_results, 
                                  ZPG, SC, CONFIG, output_dir='.')
else:
    print("Skipping save (no training history)")
    final_results = None